### GroupBy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### ```.groupby()``` : GroupBy 객체 생성

In [5]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [6]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [7]:
grouped.mean()

key1
a   -0.120101
b   -0.081206
Name: data1, dtype: float64

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.737031
      two     1.113759
b     one     0.163219
      two    -0.325632
Name: data1, dtype: float64

In [11]:
means.unstack()

key2,one,two
key1,,
a,-0.737031,1.113759
b,0.163219,-0.325632


### 

In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.113759
            2006    0.163219
Ohio        2005   -1.092158
            2006    0.384622
Name: data1, dtype: float64

In [15]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.120101,0.557198
b,-0.081206,0.041672


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.737031  0.641687
     two   1.113759  0.388222
b    one   0.163219  0.751783
     two  -0.325632 -0.668439

### 
### 그룹 간 순회
- GroupBy 객체는 이터레이션을 지원, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환

In [21]:
for name, group in df.groupby('key1') :
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.858684  1.336982
1    a  two  1.113759  0.388222
4    a  one  0.384622 -0.053608
b
  key1 key2     data1     data2
2    b  one  0.163219  0.751783
3    b  two -0.325632 -0.668439


- 색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 된다

In [25]:
for (k1, k2), group in df.groupby(['key1', 'key2']) :
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.858684  1.336982
4    a  one  0.384622 -0.053608
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.113759  0.388222
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.163219  0.751783
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.325632 -0.668439


- 그룹별 데이터를 딕셔너리로 쉽게 바꿔서 사용 가능

In [26]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.858684  1.336982
 1    a  two  1.113759  0.388222
 4    a  one  0.384622 -0.053608,
 'b':   key1 key2     data1     data2
 2    b  one  0.163219  0.751783
 3    b  two -0.325632 -0.668439}

- df의 열을 dtype에 따라 그룹으로 묶을 수도 있다.

In [27]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [37]:
grouped = df.groupby(df.dtypes, axis = 1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.858684  1.336982
1  1.113759  0.388222
2  0.163219  0.751783
3 -0.325632 -0.668439
4  0.384622 -0.053608
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 
### 열이나 열의 일부만 선택

In [39]:
df.groupby('key1')['data1'] # df['data1'].groupby(df['key1'])
df.groupby('key1')[['data2']] # df[['data2']].groupby(df['key1'])

- 소수의 열만 집계
- **data2의 컬럼에 대해서만 평균을 구하고 결과를 데이터 프레임으로 받음**

In [41]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.641687
     two   0.388222
b    one   0.751783
     two  -0.668439

- **groupby 메서드에 리스트나 배열을 넣었을 경우 DataFrameGroupBy 객체**
- **단일값으로 하나의 열 이름만 남겼을 경우, SeriesGroupBy 객체**

In [42]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [44]:
s_grouped.mean()

key1  key2
a     one     0.641687
      two     0.388222
b     one     0.751783
      two    -0.668439
Name: data2, dtype: float64

### 
### 딕셔너리와 Series에서 그룹핑

In [45]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns = ['a', 'b', 'c', 'd', 'e'],
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [53]:
people.iloc[2:3, [1, 2]] = np.nan

In [54]:
people

,a,b,c,d,e
Joe,-2.221187,-1.258102,0.275480,0.840277,1.654190
Steve,1.763703,-0.339025,-1.074199,0.138958,1.221329
Wes,1.224833,NaN,NaN,1.316402,0.222801
Jim,-0.386524,-1.613688,1.407608,-0.921935,0.488830
Travis,1.677370,-0.048050,-0.106290,-0.486372,-0.097503


In [55]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [58]:
by_column = people.groupby(mapping, axis = 'columns')
by_column.sum()

,blue,red
Joe,1.115757,-1.825099
Steve,-0.935241,2.646007
Wes,1.316402,1.447633
Jim,0.485672,-1.511383
Travis,-0.592663,1.531817


In [60]:
map_series = pd.Series(mapping)
people.groupby(map_series, axis = 'columns').count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 
### 함수로 그룹핑

- 이름의 길이별로 그룹핑

In [61]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.382878,-2.871790,1.683088,1.234743,2.365820
5,1.763703,-0.339025,-1.074199,0.138958,1.221329
6,1.677370,-0.048050,-0.106290,-0.486372,-0.097503


### 
### 인덱스 단계로 그룹핑

In [62]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names = ['cty', 'tenor'])

In [64]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.132505  1.434563  1.152221 -0.969512 -0.387493
1      0.803998 -0.129801  1.367068  0.231623  0.321584
2     -0.944571  0.478153 -0.237738 -0.038010  2.583996
3     -1.606809  1.045030 -0.420358 -0.359584 -1.177382

In [68]:
hier_df.groupby(level = 'cty', axis = 'columns').mean()

cty,JP,US
0,-0.678503,0.906430
1,0.276603,0.680422
2,1.272993,-0.234718
3,-0.768483,-0.327379


### 
### 데이터 집계
|  |  |
| -- | -- | 
| count | 그룹에서 NA가 아닌 값의 수를 반환 |
| sum | NA가 아닌 값들의 합을 계산 |
| mean | NA가 아닌 값들의 평균을 계산 |
| median | NA가 아닌 값들의 산술 중간값을 계산 |
| std, var | 편향되지 않은 (n-1을 분모로 하는) 표준편차와 분산 |
| min, max | NA가 아닌 값들 중 최솟값과 최댓값 |
| prod | NA가 아닌 값들의 곱 |
| first, last | NA가 아닌 값들 중 첫째 값과 마지막 값 |

In [69]:
df

,key1,key2,data1,data2
0,a,one,-1.858684,1.336982
1,a,two,1.113759,0.388222
2,b,one,0.163219,0.751783
3,b,two,-0.325632,-0.668439
4,a,one,0.384622,-0.053608


In [71]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.967931
b    0.114334
Name: data1, dtype: float64

### ```grouped.agg(사용자 함수)```

In [72]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [73]:
grouped.agg(peak_to_peak)

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_19744\4193407341.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.972443,1.390590
b,0.488851,1.420222


- describe 메서드는 데이터를 집계하지 않아도 잘 작동

In [74]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.120101  1.549165 -1.858684 -0.737031  0.384622  0.749190   
b      2.0 -0.081206  0.345670 -0.325632 -0.203419 -0.081206  0.041006   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.113759   3.0  0.557198  0.710528 -0.053608  0.167307  0.388222   
b     0.163219   2.0  0.041672  1.004249 -0.668439 -0.313384  0.041672   

                          
           75%       max  
key1                      
a     0.862602  1.336982  
b     0.396727  0.751783

### 
### 열에 여러 가지 함수 적용

In [90]:
tips = pd.read_csv("examples/tips.csv")

In [91]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [92]:
tips.head(3)

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587


In [93]:
grouped = tips.groupby(['day', 'smoker'])

In [97]:
grouped_pct = grouped['tip_pct']

In [98]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

- **이름과 함수가 담긴 (name, function) 튜플의 리스트 적용**

In [99]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

- **열마다 다른 함수를 적용하거나, 여러 개의 함수를 모든 열에 적용**

In [100]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_19744\576261660.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [101]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

- **열 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능**

In [102]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_19744\3909980601.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

- **열마다 다른 함수를 적용하고 싶다면, agg 메서드에 열 이름에 대응하는 함수가 들어있는 딕셔너리를 넘김**

In [104]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 인덱싱되지 않은 형태로 집계된 데이터 반환
- groupby 메서드에 ```as_index = False```를 넘겨 인덱싱되지 않도록 설정

In [105]:
tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### 
### Apply : 일반적인 분리-적용-병합

In [106]:
def top(df, n = 5, column = 'tip_pct') :
    return df.sort_values(by = column)[-n:]

In [107]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


- **흡연자(smoker)그룹에 대해 top 함수를 apply**

In [108]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

- **smoker과 day 그룹별, total_bill이 가장 높은 값 계산**

In [109]:
tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### 
### 그룹 인덱스 생략
- groupby 메서드에 ```group_keys = False```를 넘김

In [110]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 
### 변위치 분석과 버킷 분석
- ```cut```과 ```qcut```메서드를 groupby와 조합

### 등간격 버킷 분석

In [111]:
frame = pd.DataFrame({'data1' : np.random.randn(1000),
                      'data2' : np.random.randn(1000)})

In [112]:
quantiles = pd.cut(frame.data1, 4)

In [114]:
quantiles[:10]

0    (-1.535, -0.0329]
1    (-1.535, -0.0329]
2     (-3.044, -1.535]
3    (-1.535, -0.0329]
4    (-1.535, -0.0329]
5     (-0.0329, 1.469]
6    (-1.535, -0.0329]
7     (-0.0329, 1.469]
8    (-1.535, -0.0329]
9     (-0.0329, 1.469]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.044, -1.535] < (-1.535, -0.0329] < (-0.0329, 1.469] < (1.469, 2.972]]

- cut에서 반환된 Categorical 객체는 바로 groupby로 넘길 수 있다.

- **frame의 data2의 범주별 min, max, count, mean**

In [116]:
def get_stats(group) :
    return {'min' : group.min(), 'max' : group.max(),
            'count' : group.count(), 'mean' : group.mean()}

In [117]:
grouped = frame.data2.groupby(quantiles)

In [118]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.044, -1.535]",-2.035258,2.637639,63.0,0.031199
"(-1.535, -0.0329]",-3.165691,2.609940,398.0,-0.045119
"(-0.0329, 1.469]",-2.655465,3.011756,476.0,0.001481
"(1.469, 2.972]",-2.771124,2.204260,63.0,-0.088395


- **표본 변위치에 기반하여 크기가 같은 버킷을 계산하려면 qcut을 사용**
- ```labels = False```를 넘겨 변위치 숫자 계산

In [120]:
grouping = pd.qcut(frame.data1, 10, labels = False)

In [122]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.112804,2.637639,100.0,0.053672
1,-3.165691,2.609940,100.0,-0.001966
2,-2.840543,2.511695,100.0,-0.058997
3,-2.496537,1.827906,100.0,-0.135116
4,-2.795125,2.577617,100.0,-0.092283
5,-2.374318,2.330124,100.0,-0.071881
6,-2.655465,2.070124,100.0,0.117211
7,-2.237200,1.961556,100.0,-0.034578
8,-1.974382,3.011756,100.0,0.037235


### 
### 예제 : 그룹에 따른 결측치 채우기

In [123]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

In [126]:
s.fillna(s.mean()) # 결측값을 평균값으로 대체

0    0.589969
1    0.084778
2    0.589969
3    1.056932
4    0.589969
5    0.628198
dtype: float64

In [128]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states)
data

Ohio         -1.612510
New York     -0.116236
Vermont       2.821798
Florida       1.693711
Oregon       -1.418000
Nevada        2.103426
California    0.021918
Idaho         0.433714
dtype: float64

In [129]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.612510
New York     -0.116236
Vermont            NaN
Florida       1.693711
Oregon       -1.418000
Nevada             NaN
California    0.021918
Idaho              NaN
dtype: float64

In [130]:
data.groupby(group_key).mean()

East   -0.011678
West   -0.698041
dtype: float64

- **해당 관측값이 속한 그룹의 평균으로 결측치를 대체**

In [131]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean) 

Ohio         -1.612510
New York     -0.116236
Vermont      -0.011678
Florida       1.693711
Oregon       -1.418000
Nevada       -0.698041
California    0.021918
Idaho        -0.698041
dtype: float64

- **해당 관측값이 속한 그룹에 해당하는 값으로 결측치를 대체 (동부는 0.5, 서부는 -1)**

In [132]:
fill_values = {'East' : 0.5, 'West' : -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.612510
New York     -0.116236
Vermont       0.500000
Florida       1.693711
Oregon       -1.418000
Nevada       -1.000000
California    0.021918
Idaho        -1.000000
dtype: float64

### 
### 예제 : 랜덤 표본과 순열

- 트럼프 카드 덱 생성

In [162]:
# 하트, 스페이드, 클럽, 다이아몬드
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

- deck에서 5장의 카드를 뽑는 draw

In [139]:
def draw(deck, n = 5):
    return deck.sample(n)

draw(deck)

10C    10
8S      8
5H      5
3S      3
9D      9
dtype: int64

- 각 세트(하트, 스페이드, 클럽, 다이아몬드)별로 2장의 카드를 무작위로 뽑음
- 세트는 각 카드 이름의 마지막 글자이므로, 이를 이용하여 그룹을 나누고 apply를 사용

In [140]:
get_suit = lambda card: card[-1] # 마지막 글자가 세트
deck.groupby(get_suit).apply(draw, n = 2)

C  8C      8
   3C      3
D  9D      9
   QD     10
H  9H      9
   10H    10
S  QS     10
   6S      6
dtype: int64

In [141]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

10C    10
2C      2
JD     10
8D      8
10H    10
7H      7
KS     10
3S      3
dtype: int64

### 
### 예제 : 그룹 가중 평균과 상관관계

In [163]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.277331,0.399418
1,a,-0.221696,0.070455
2,a,0.644852,0.999708
3,a,-0.662725,0.741996
4,b,0.074195,0.645565
5,b,-1.442807,0.697061
6,b,-0.014952,0.863191
7,b,0.953977,0.928632


In [167]:
grouped = df.groupby('category')
get_wavg = lambda g : np.average(g['data'], weights = g['weights'])
grouped.apply(get_wavg)

category
a    0.011998
b   -0.027067
dtype: float64

### 

In [168]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates = True,
                       index_col = 0)

In [174]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [178]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


- **SPX열과 다른 열의 상관관계 계산**

In [179]:
spx_corr = lambda x : x.corrwith(x['SPX'])

- **퍼센트 변화율 계산**

### - ```.pct_change()``` : 퍼센트 변화율 계산

In [180]:
rets = close_px.pct_change().dropna()

- **각 datetime에서 연도 속성만 반환 후, SPX열과의 상관관계 계산**

In [181]:
get_year = lambda x : x.year

In [187]:
by_year = rets.groupby(get_year)

In [183]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


- **두 열 간의 상관관계 계산**

In [184]:
by_year.apply(lambda g : g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 
### 예제 : 그룹상의 선형회귀

In [191]:
import statsmodels.api as sm

In [192]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [193]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### 
### 피벗테이블과 교차일람표
- 피벗테이블 : 스프레드시트 프로그램과 그 외 다른 데이터 분석 소프트웨어에서 흔히 볼 수 있는 데이터 요약화 도구

### ```pd.pivot_table()```
- 마진이라고 하는 부분합을 추가할 수 있는 기능 제공

#### pivot_table 메서드 옵션
|  |  |
| -- | -- |
| values | 집계하려는 열 혹은 이름의 리스트. 기본적으로 모든 숫자 열을 집계 |
| index | 만들어지는 피벗테이블의 행을 그룹으로 묶을 행 이름이나 그룹 키 |
| columns | 만들어지는 피벗테이블의 열을 그룹으로 묶을 열 이름이나 그룹 키 |
| aggfunc | 집계함수나 함수 리스트, 기본값으로 'mean'이 사용. groupby 컨텍스트 안에서 유효한 어떤 함수라도 가능 |
| fill_value | 결과 테이블에서 누락된 값을 대체하기 위한 값 |
| dropna | True인 경우 모든 항목이 NA인 열은 포함하지 않음 |
| margins | 부분합이나 총계를 담기 위한 행 / 열을 추가할지 여부 (기본값은 False) |

- **day와 smoker 집단에서 평균(pivot_table의 기본 연산) 계산**

In [194]:
tips.pivot_table(index = ['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

- **tip_pct와 size에 대해서만 집계, time별 그룹핑**

In [196]:
tips.pivot_table(['tip_pct', 'size'], 
                 index = ['time', 'day'],
                 columns = 'smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

- **```margins = True```를 추가하여 부분합 포함**

In [197]:
tips.pivot_table(['tip_pct', 'size'], 
                 index = ['time', 'day'],
                 columns = 'smoker', margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

- **```aggfunc```로 다른 집계함수를 사용**

In [198]:
tips.pivot_table('tip_pct', 
                 index = ['time', 'smoker'], 
                 columns = 'day',
                 aggfunc = len, margins = True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

- **만약 어떤 조합이 비어있다면 (결측치 존재) ```fill_value```를 사용**

In [199]:
tips.pivot_table('tip_pct', 
                 index = ['time', 'size', 'smoker'],
                 columns = 'day', 
                 aggfunc = 'mean', 
                 fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 
### 교차일람표 (교차표)

In [201]:
from io import StringIO

In [202]:
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

### ```pd.crosstab(x, y, margins)``` : 교차표 생성

In [204]:
pd.crosstab(data.Nationality, data.Handedness, margins = True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [205]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244